In [6]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from copy import deepcopy
import random
from sklearn.utils import shuffle

In [7]:
#read data
data=pd.read_table("./features.txt",sep="\t")

In [8]:
data

,TextID,URL,Label,totalWordsCount,semanticobjscore,semanticsubjscore,CC,CD,DT,EX,...,pronouns2nd,pronouns3rd,compsupadjadv,past,imperative,present3rd,present1st2nd,sentence1st,sentencelast,txtcomplexity
0,Text0001,http://msn.foxsports.com/foxsoccer/mexico/stor...,objective,109,0,1,7,9,0,5,...,0,3,0,11,0,0,0,0,1,18
1,Text0002,http://msn.foxsports.com/foxsoccer/premierleag...,objective,309,21,4,1,19,1,4,...,0,10,0,13,0,14,9,1,1,14
2,Text0003,http://uk.eurosport.yahoo.com/04022011/58/fed-...,objective,149,6,1,8,14,0,5,...,0,2,0,8,0,3,2,1,1,18
3,Text0004,http://uk.eurosport.yahoo.com/07022011/58/bund...,objective,305,18,5,7,26,0,10,...,0,8,3,13,1,7,1,1,1,20
4,Text0005,http://uk.eurosport.yahoo.com/05022011/58/fed-...,objective,491,23,8,33,47,0,12,...,0,16,2,34,1,5,6,1,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Text0996,http://www.nba.com/pistons/news/blaha_090313.html,subjective,926,46,34,5,83,1,20,...,16,46,10,13,12,49,29,1,1,24
996,Text0997,http://www.nba.com/pistons/news/smashing_succe...,subjective,876,48,26,9,109,1,16,...,8,43,5,34,9,19,31,1,1,21
997,Text0998,http://www.foxsportswest.com/01/07/13/Hypocrit...,subjective,1469,82,53,14,171,1,10,...,9,49,12,40,19,51,42,1,1,18
998,Text0999,http://www.football365.com/f365-features/84318...,subjective,343,7,5,4,24,0,9,...,0,8,1,3,3,25,7,1,1,11


In [9]:
features=data.columns.drop(["Label","WRB","NNP","TextID","URL","totalWordsCount","semanticobjscore","semanticsubjscore"])

In [10]:
features

Index(['CC', 'CD', 'DT', 'EX', 'FW', 'INs', 'JJ', 'JJR', 'JJS', 'LS', 'MD',
       'NN', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS',
       'RP', 'SYM', 'TOs', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',
       'WDT', 'WP', 'WP$', 'baseform', 'Quotes', 'questionmarks',
       'exclamationmarks', 'fullstops', 'commas', 'semicolon', 'colon',
       'ellipsis', 'pronouns1st', 'pronouns2nd', 'pronouns3rd',
       'compsupadjadv', 'past', 'imperative', 'present3rd', 'present1st2nd',
       'sentence1st', 'sentencelast', 'txtcomplexity'],
      dtype='object')

In [11]:
#Maximum and minimum normalization
def MaxMinNormalization(x):
    x = (x - np.min(x.values)) / (np.max(x.values) - np.min(x.values))
    return x
 
for c in features:
    data[c]=MaxMinNormalization(data[c])

In [12]:
data

,TextID,URL,Label,totalWordsCount,semanticobjscore,semanticsubjscore,CC,CD,DT,EX,...,pronouns2nd,pronouns3rd,compsupadjadv,past,imperative,present3rd,present1st2nd,sentence1st,sentencelast,txtcomplexity
0,Text0001,http://msn.foxsports.com/foxsoccer/mexico/stor...,objective,109,0,1,0.031111,0.019108,0.000000,0.052083,...,0.000000,0.023622,0.000000,0.050000,0.000000,0.000000,0.000000,0.0,1.0,0.191176
1,Text0002,http://msn.foxsports.com/foxsoccer/premierleag...,objective,309,21,4,0.004444,0.040340,0.083333,0.041667,...,0.000000,0.078740,0.000000,0.059091,0.000000,0.098592,0.092784,1.0,1.0,0.132353
2,Text0003,http://uk.eurosport.yahoo.com/04022011/58/fed-...,objective,149,6,1,0.035556,0.029724,0.000000,0.052083,...,0.000000,0.015748,0.000000,0.036364,0.000000,0.021127,0.020619,1.0,1.0,0.191176
3,Text0004,http://uk.eurosport.yahoo.com/07022011/58/bund...,objective,305,18,5,0.031111,0.055202,0.000000,0.104167,...,0.000000,0.062992,0.076923,0.059091,0.032258,0.049296,0.010309,1.0,1.0,0.220588
4,Text0005,http://uk.eurosport.yahoo.com/05022011/58/fed-...,objective,491,23,8,0.146667,0.099788,0.000000,0.125000,...,0.000000,0.125984,0.051282,0.154545,0.032258,0.035211,0.061856,1.0,1.0,0.279412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Text0996,http://www.nba.com/pistons/news/blaha_090313.html,subjective,926,46,34,0.022222,0.176221,0.083333,0.208333,...,0.363636,0.362205,0.256410,0.059091,0.387097,0.345070,0.298969,1.0,1.0,0.279412
996,Text0997,http://www.nba.com/pistons/news/smashing_succe...,subjective,876,48,26,0.040000,0.231423,0.083333,0.166667,...,0.181818,0.338583,0.128205,0.154545,0.290323,0.133803,0.319588,1.0,1.0,0.235294
997,Text0998,http://www.foxsportswest.com/01/07/13/Hypocrit...,subjective,1469,82,53,0.062222,0.363057,0.083333,0.104167,...,0.204545,0.385827,0.307692,0.181818,0.612903,0.359155,0.432990,1.0,1.0,0.191176
998,Text0999,http://www.football365.com/f365-features/84318...,subjective,343,7,5,0.017778,0.050955,0.000000,0.093750,...,0.000000,0.062992,0.025641,0.013636,0.096774,0.176056,0.072165,1.0,1.0,0.088235


In [13]:
data["Label"].value_counts()

objective     635
subjective    365
Name: Label, dtype: int64

In [14]:
data['Label'].replace(("objective","subjective"),(0,1),inplace=True)

In [15]:
data = shuffle(data)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 915 to 810
Data columns (total 62 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   TextID             1000 non-null   object 
 1   URL                1000 non-null   object 
 2   Label              1000 non-null   int64  
 3   totalWordsCount    1000 non-null   int64  
 4   semanticobjscore   1000 non-null   int64  
 5   semanticsubjscore  1000 non-null   int64  
 6   CC                 1000 non-null   float64
 7   CD                 1000 non-null   float64
 8   DT                 1000 non-null   float64
 9   EX                 1000 non-null   float64
 10  FW                 1000 non-null   float64
 11  INs                1000 non-null   float64
 12  JJ                 1000 non-null   float64
 13  JJR                1000 non-null   float64
 14  JJS                1000 non-null   float64
 15  LS                 1000 non-null   float64
 16  MD                 1000

In [17]:


"""
    BP neural network model. The weights between layers are randomly generated and obey the standard normal distribution.
"""

def Sigmoid(x):
    """
    calculation formula of the S-type activation function.
    :param x: Data to be calculated.
    :return: Function value of S-type activation function.
    """
    function = 1.0 / (1.0 + np.exp(-x))
    return function

def Sigmoid_Derivative(x):
    """
    derivative calculation formula of the S-type activation function.
    
    """
    f = Sigmoid(x)
    derivative = f*(1.0-f)
    return derivative

class BPNN:
    def __init__(self,input_n,hidden_n,output_n,input_weights = None,hidden_weights = None,
                 hidden_threshold = None,output_threshold = None):
        """
        constructor of the BP neural network class.
        :param input_n: Number of neurons in the input layer.
        :param hidden_n: Number of hidden layer neurons.
        :param output_n: Number of neurons in the output layer.
        """
        self.Train_Data = []                                                                        # 训练数据集
        self.Train_Label = []                                                                       # 训练数据集标签
        self.input_n = input_n                                                                      # 输入层神经元个数
        self.hidden_n = hidden_n                                                                    # 隐含层神经元个数
        self.output_n = output_n                                                                    # 输出层神经元个数
        self.input_cells = np.zeros(self.input_n).reshape((1,self.input_n))                         # 输入层神经元
        self.hidden_cells = np.zeros(self.hidden_n).reshape((1,self.hidden_n))                      # 隐含层神经元
        self.hidden_cells_input = np.zeros(self.hidden_n).reshape((1,self.hidden_n))                # 隐含层的输入（不含阈值后进行sigmoid）
        self.output_cells = np.zeros(self.output_n).reshape((1,self.output_n))                      # 输出层神经元
        self.output_cells_input = np.zeros(self.output_n).reshape((1,self.output_n))                # 输出层的输入（不含阈值后进行sigmoid）
        if input_weights is not None:
            self.input_weights = input_weights
        else:
            self.input_weights = np.random.randn(self.input_n,self.hidden_n)                        # 输入层与隐含层之间的权重
        if hidden_weights is not None:
            self.hidden_weights = hidden_weights
        else:
            self.hidden_weights = np.random.randn(self.hidden_n,self.output_n)                      # 隐含层与输出层之间的权重
        if hidden_threshold is not None:
            self.hidden_threshold = hidden_threshold
        else:
            self.hidden_threshold = np.random.randn(1, self.hidden_n)                               # 隐含层的阈值
        if output_threshold is not None:
            self.output_threshold = output_threshold
        else:
            self.output_threshold = np.random.randn(1, self.output_n)                               # 输出层的阈值
        self.input_weights_copy = deepcopy(self.input_weights)                                      # 输入层与隐含层之间的权重备份
        self.hidden_weights_copy = deepcopy(self.hidden_weights)                                    # 隐含层与输出层之间的权重备份
        self.hidden_threshold_copy = deepcopy(self.hidden_threshold)                                # 隐含层的阈值备份
        self.output_threshold_copy = deepcopy(self.output_threshold)                                # 输出层的阈值备份

    def Init(self,Train_Data,Train_Label):
        """
        Function to initialize training data and labels
        """
        self.Train_Data = Train_Data
        self.Train_Label = Train_Label

    def predict(self,input):
        """
        BP neural network forward learning transfer function
        """
        # Copy input layer input data
        self.input_cells = deepcopy(input)
        # Calculate hidden layer output
        self.hidden_cells_input = self.input_cells.dot(self.input_weights)
        self.hidden_cells = self.hidden_cells_input + self.hidden_threshold
        self.hidden_cells = Sigmoid(self.hidden_cells)
        # Calculate output layer output
        self.output_cells_input = self.hidden_cells.dot(self.hidden_weights)
        self.output_cells = self.output_cells_input + self.output_threshold
        self.output_cells = Sigmoid(self.output_cells)
        return self.output_cells

    def Gradient_Vector(self,ideal_output):
        """
        Function to calculate the relevant gradient vector of each set of input data
        :param ideal_output: target or actual real output
        """
        # The updated relative gradient of the weight and threshold between the hidden layer and the output layer
        error = ideal_output - self.output_cells
        derivative = Sigmoid_Derivative(self.output_cells)
        g = derivative * error
        # Hidden layer-output layer weight increment
        hidden_weights_increasement = self.hidden_cells.T.dot(g)
        # Hidden layer threshold increment
        output_threshold_increasement = -g
        # 
        e = Sigmoid_Derivative(self.hidden_cells) * g.dot(self.hidden_weights.T)
        # Input layer-hidden layer weight increment
        input_weights_increasement = self.input_cells.T.dot(e)
        # Input layer threshold increment
        hidden_threshold_increasement = -e
        return hidden_weights_increasement,output_threshold_increasement\
            ,input_weights_increasement,hidden_threshold_increasement

    def back_propagate(self,hidden_weights_increasement,output_threshold_increasement\
            ,input_weights_increasement,hidden_threshold_increasement,learn_rate):
        """
        Backward error transfer function, parameter adjustment
        :param g: The updated relative gradient of the weight and threshold between the hidden layer and the output layer
        :param e: The updated relative gradient of the weight and threshold between the input layer and the output layer
        :param input_cell: The sum of the input layer
        :param hidden_cell: The sum of hidden layers
        """
        # Update of the weight between the hidden layer and the output layer
        self.hidden_weights = self.hidden_weights + hidden_weights_increasement*learn_rate
        #Threshold update of the output layer
        self.output_threshold = self.output_threshold + output_threshold_increasement*learn_rate
        #Weight update between input layer and hidden layer
        self.input_weights = self.input_weights + learn_rate*input_weights_increasement
        #Hidden layer threshold update
        self.hidden_threshold = self.hidden_threshold + hidden_threshold_increasement*learn_rate

    def reset(self):
        """
        reset function of BPNN, which restores the initial values of related parameters after changing an iteration process
        """
        self.input_weights = deepcopy(self.input_weights_copy)  # Weight backup between input layer and hidden layer
        self.hidden_weights = deepcopy(self.hidden_weights_copy)  # Weight backup between hidden layer and output layer
        self.hidden_threshold = deepcopy(self.hidden_threshold_copy)  # Threshold backup of hidden layer
        self.output_threshold = deepcopy(self.output_threshold_copy)  # Threshold backup of the output layer

    def train_batch(self,input,output):
        """
        Function to train on one set of data at a time
        
        """
        # predicted classification result
        
        output = output.reshape(1, len(output))
        input = input.reshape(1, len(input))
        self.output_cells = self.predict(input)
        # Calculate the relevant gradient vector
        hidden_weights_increasement, output_threshold_increasement \
            , input_weights_increasement, hidden_threshold_increasement = self.Gradient_Vector(output)
        # Calculate classification error
        error = np.sum((output-self.output_cells)**2)
        return hidden_weights_increasement,output_threshold_increasement\
            ,input_weights_increasement,hidden_threshold_increasement,error

    def train_BGD(self,inputs,outputs,limitation,learn_rate):
        """
        BP neural network training function, using BGD algorithm for parameter update
        
        :param limitaion: Number of iterations
        
        """
        #Initialize the training data set and the corresponding label
        self.Init(inputs,outputs)
        for j in range(limitation):
            self.train_dataset_BGD(learn_rate)

    def train_dataset_BGD(self,learn_rate):
        """
        The function of using the BGD algorithm to train the entire training set in one iteration
        
        """
        Hidden_Weights_Increasement = []                                # Hidden layer-weight increment between output layer
        Output_Threshold_Increasement = []                              # Threshold increment between output layers
        Input_Weights_Increasement = []                                 # Weight increment between input layer and hidden layer
        Hidden_Threshold_Increasement = []                              # Hidden layer threshold increment
        for (train_data,train_label) in zip(self.Train_Data,self.Train_Label):
            # Traverse and train, and get the relevant gradient increment and the corresponding error
            hidden_weights_increasement, output_threshold_increasement \
            ,input_weights_increasement, hidden_threshold_increasement,err = self.train_batch(train_data,train_label)
            Hidden_Weights_Increasement.append(hidden_weights_increasement)
            Output_Threshold_Increasement.append(output_threshold_increasement)
            Input_Weights_Increasement.append(input_weights_increasement)
            Hidden_Threshold_Increasement.append(hidden_threshold_increasement)
        # Calculate the total gradient
        hidden_weights_increasement_sum = np.sum(np.array(Hidden_Weights_Increasement),0)
        output_threshold_increasement_sum = np.sum(np.array(Output_Threshold_Increasement),0)
        input_weights_increasement_sum = np.sum(np.array(Input_Weights_Increasement),0)
        hidden_threshold_increasement_sum = np.sum(np.array(Hidden_Threshold_Increasement),0)
        # Perform error back propagation and adjust hyperparameters
        self.back_propagate(hidden_weights_increasement_sum,output_threshold_increasement_sum,
                            input_weights_increasement_sum,hidden_threshold_increasement_sum,learn_rate)

    def train_SGD(self,inputs,outputs,limitation,learn_rate):
        """
        Training function of BP neural network, using SGD algorithm to update parameters
        
        """
        #Initialize the training data set and the corresponding label
        self.Init(inputs,outputs)
        for j in range(limitation):
            self.train_dataset_SGD(learn_rate)

    def train_dataset_SGD(self, learn_rate):
        """
        A function to train the entire training set using the SGD algorithm in one iteration
        
        """
        # Randomly shuffle the data set and label set
        self.Shuffle_SGD()
        for (train_data, train_label) in zip(self.Train_Data, self.Train_Label):
            # 
            hidden_weights_increasement, output_threshold_increasement, input_weights_increasement, \
                    hidden_threshold_increasement, err = self.train_batch(train_data,train_label)
            # Perform error back propagation and adjust hyperparameters
            self.back_propagate(hidden_weights_increasement, output_threshold_increasement,
                                input_weights_increasement, hidden_threshold_increasement, learn_rate)

    def Shuffle_SGD(self):
        """
        Function to randomly shuffle the data set and the mark set before executing the SGD algorithm
        """
        #First generate a sequence of natural numbers with the same length as the training data for the SGD algorithm and scramble
        length = len(self.Train_Label)
        random_sequence = list(np.arange(length))
        random.shuffle(random_sequence)
        data = [self.Train_Data[index] for index in random_sequence]
        label = [self.Train_Label[index] for index in random_sequence]
        self.Train_Data = np.array(data)
        self.Train_Label = np.array(label)

    def train_MBGD(self,inputs,outputs,limitation,learn_rate,mini_batch_size):
        """
        using MBGD algorithm for parameter update
        
        """
        
        self.Init(inputs,outputs)
        for j in range(limitation):
            self.train_dataset_MBGD(learn_rate,mini_batch_size)

    def train_dataset_MBGD(self, learn_rate,mini_batch_size):
        """
        A function to train the entire training set using the MBGD algorithm in one iteration
        """
        
        self.Split_MBGD(mini_batch_size)
        for (mini_batch_data, mini_batch_label) in zip(self.Train_Data, self.Train_Label):
            Hidden_Weights_Increasement = []  # Hidden layer-weight increment between output layer
            Output_Threshold_Increasement = []  # Threshold increment between output layers
            Input_Weights_Increasement = []  # Weight increment between input layer and hidden layer
            Hidden_Threshold_Increasement = []  # Hidden layer threshold increment
            # Traverse and train, and get the relevant gradient increment and the corresponding error
            for (train_data,train_label) in zip(mini_batch_data, mini_batch_label):
                hidden_weights_increasement, output_threshold_increasement, input_weights_increasement, \
                        hidden_threshold_increasement, err = self.train_batch(train_data,train_label)
                Hidden_Weights_Increasement.append(hidden_weights_increasement)
                Output_Threshold_Increasement.append(output_threshold_increasement)
                Input_Weights_Increasement.append(input_weights_increasement)
                Hidden_Threshold_Increasement.append(hidden_threshold_increasement)
            # Calculate the gradient average
            hidden_weights_increasement_avg = np.average(np.array(Hidden_Weights_Increasement), 0)
            output_threshold_increasement_avg = np.average(np.array(Output_Threshold_Increasement), 0)
            input_weights_increasement_avg = np.average(np.array(Input_Weights_Increasement), 0)
            hidden_threshold_increasement_avg = np.average(np.array(Hidden_Threshold_Increasement), 0)
            # Perform error back propagation and adjust hyperparameters
            self.back_propagate(hidden_weights_increasement_avg, output_threshold_increasement_avg,
                                input_weights_increasement_avg, hidden_threshold_increasement_avg, learn_rate)
        self.Return()

    def Split_MBGD(self,mini_batch_size):
        """
        Before executing the MBGD algorithm, a function to generate mini-batches from the data set and the marker set
        """
        '''
        Since it is not known whether the length of the data set can be divisible by mini_batch_size,
        First calculate the remainder rest of the data set length and mini_batch_size,
        If rest is zero, it is better not to execute the following if judgment statement,
        If it is not zero, first allocate the data set without the remainder,
        Then add the remaining data and results through the following if statement.
       '''
        length = len(self.Train_Label)
        rest = length % mini_batch_size
        random_sequence = np.arange(length)
        random.shuffle(random_sequence)
        mini_batches = [random_sequence[k:k + mini_batch_size] for k in
                        range(0, length - rest, mini_batch_size)]
        mini_batch_inputs = []
        mini_batch_outputs = []
        for batch in mini_batches:
            mini_batch_data = [self.Train_Data[_batch] for _batch in batch]
            mini_batch_label = [self.Train_Label[_batch] for _batch in batch]
            mini_batch_inputs.append(list(mini_batch_data))
            mini_batch_outputs.append(list(mini_batch_label))
        # The length of the data set cannot be divisible by mini_batch_size, and the remaining data forms the last mini_batch
        if rest > 0:
            batch = random_sequence[length - rest:length]
            mini_batch_data = [self.Train_Data[_batch] for _batch in batch]
            mini_batch_label = [self.Train_Label[_batch] for _batch in batch]
            mini_batch_inputs.append(list(mini_batch_data))
            mini_batch_outputs.append(list(mini_batch_label))
        self.Train_Data = np.array(mini_batch_inputs)
        self.Train_Label = np.array(mini_batch_outputs)

    def Return(self):
        """
        The function to restore the small sample data set and label set of the MBGD algorithm
        """
        data = []
        label = []
        for (train_data,train_label) in zip(self.Train_Data,self.Train_Label):
            for (_data,_label) in zip(train_data,train_label):
                data.append(_data)
                label.append(_label)
        self.Train_Data = np.array(data)
        self.Train_Label = np.array(label)

    def test(self,Test_Data):
        """
        BP neural network test function
        """
        predict_labels = []
        # Generate standard output neurons and set 0
        tmp = [0]*self.output_n
        for test_data in Test_Data:
            predict_output = self.predict(test_data)
            index = np.argmax(predict_output)
            tmp1 = deepcopy(tmp)
            tmp1[index] = 1
            predict_labels.append(tmp1)
        predict_labels = np.array(predict_labels)
        return predict_labels


In [18]:
def Merge(data,columns):
    """
    This is the function to merge arrays into DataFrame
    
    """
    Data = np.array(data)
    Data = Data.T
    ans = pd.DataFrame(data=Data,columns=columns)
    return ans

In [19]:
Data=np.array(data[features])
Label=[]
for i in data["Label"].values:
    if i==0:
        Label.append([1.0,0.0])
    else:
        Label.append([0.0,1.0])
Label=np.array(Label)

In [20]:
Label

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [21]:
Data

array([[0.01777778, 0.17622081, 0.41666667, ..., 1.        , 1.        ,
        0.16176471],
       [0.19555556, 0.21019108, 0.08333333, ..., 1.        , 1.        ,
        0.19117647],
       [0.10666667, 0.16135881, 0.        , ..., 1.        , 1.        ,
        0.19117647],
       ...,
       [0.06222222, 0.15498938, 0.08333333, ..., 1.        , 1.        ,
        0.25      ],
       [0.07111111, 0.16985138, 0.16666667, ..., 1.        , 1.        ,
        0.16176471],
       [0.16      , 0.1910828 , 0.08333333, ..., 1.        , 1.        ,
        0.19117647]])

In [ ]:
best_accr=0
best_model=0
mpl.rcParams['font.sans-serif'] = [u'simHei']
mpl.rcParams['axes.unicode_minus'] = False

# Initialize the array of iteration times, small sample size, L2 regularization parameter, learning rate array
learn_rate = 0.01
Iteration = 800
mini_batch = 64

# Initialize BP neural network and interstitial BP neural network and Brain_Interstitial_BPNN model
col1 = np.shape(Data)[1]
col2 = np.shape(Label)[1]
input_n = col1
hidden_n = int(np.sqrt(col1 * col2))
output_n = col2

# 
BGD_Accuracy = []               # Average accuracy of BGD algorithm
SGD_Accuracy = []               # SGD algorithm average accuracy
MBGD_Accuracy = []              # Average accuracy of MBGD algorithm
BGD_MSE = []                    # BGD algorithm MSE
SGD_MSE = []                    # SGD algorithm MSE
MBGD_MSE = []                   # MBGD algorithm MSE

# 
kf = KFold(n_splits=5, shuffle=True, random_state=np.random.randint(0, len(Data)))
train_test_index = []
for (train_index, test_index) in kf.split(Data):
    train_test_index.append((train_index, test_index))
for (train_index, test_index) in train_test_index:
    # Divide the data set into training data set and test data set
    Train_Data = Data[train_index]
    Train_Label = Label[train_index]
    Test_Data = Data[test_index]
    Test_Label = Label[test_index]

    # Initialize the related result array
    bgd_accuracy = []                           # One-time cross-validation accuracy of bgd algorithm
    sgd_accuracy = []                           # One-time cross validation accuracy of sgd algorithm
    mbgd_accuracy = []                          # One-time cross validation accuracy of mbgd algorithm
    bgd_mse = []                                # One-time cross-validation MSE of bgd algorithm
    sgd_mse = []                                # One-time cross-validation MSE of sgd algorithm
    mbgd_mse = []                               # One-time cross-validation MSE of mbgd algorithm

    # Initialize BPNN of various gradient algorithms
    BGD = BPNN(input_n, hidden_n, output_n)
    SGD = BPNN(input_n, hidden_n, output_n)
    MBGD = BPNN(input_n, hidden_n, output_n)
    BGD.Init(Train_Data,Train_Label)
    SGD.Init(Train_Data, Train_Label)
    MBGD.Init(Train_Data, Train_Label)

    for iter in np.arange(Iteration):
        BGD.train_dataset_BGD(learn_rate)
        SGD.train_dataset_SGD(learn_rate)
        MBGD.train_dataset_MBGD(learn_rate,mini_batch)
        # BGD Algorithm result
        predict_outputs = BGD.test(Test_Data)
        bgd_accuracy.append(accuracy_score(Test_Label, predict_outputs))
        bgd_mse.append(np.sum((Test_Label - predict_outputs) ** 2) / len(Test_Label))
        if accuracy_score(Test_Label, predict_outputs)>best_accr:
            best_accr=accuracy_score(Test_Label, predict_outputs)
            best_model=BGD
        # SGD Algorithm result
        predict_outputs = SGD.test(Test_Data)
        sgd_accuracy.append(accuracy_score(Test_Label, predict_outputs))
        sgd_mse.append(np.sum((Test_Label - predict_outputs) ** 2) / len(Test_Label))
        if accuracy_score(Test_Label, predict_outputs)>best_accr:
            best_accr=accuracy_score(Test_Label, predict_outputs)
            best_model=SGD
        # MBGD Algorithm result
        predict_outputs = BGD.test(Test_Data)
        mbgd_accuracy.append(accuracy_score(Test_Label, predict_outputs))
        mbgd_mse.append(np.sum((Test_Label - predict_outputs) ** 2) / len(Test_Label))
        if accuracy_score(Test_Label, predict_outputs)>best_accr:
            best_accr=accuracy_score(Test_Label, predict_outputs)
            best_model=MBGD
        if (iter+1) % 10 == 0:
            print("%d,learn_rate=%f" % (iter + 1,learn_rate))
            print("BGD TA: %f,MSE：%f" % (bgd_accuracy[iter], bgd_mse[iter]))
            print("SGD TA: %f,MSE：%f" % (sgd_accuracy[iter], sgd_mse[iter]))
            print("MBGD TA：%f,MSE：%f" % (mbgd_accuracy[iter], mbgd_mse[iter]))
    BGD_Accuracy.append(list(bgd_accuracy))
    SGD_Accuracy.append(list(sgd_accuracy))
    MBGD_Accuracy.append(list(mbgd_accuracy))
    BGD_MSE.append(list(bgd_mse))
    SGD_MSE.append(list(sgd_mse))
    MBGD_MSE.append(list(mbgd_mse))

# Calculate and visualize the average test mean square error of the BP neural network model and the neural network model based on the intercellular environment
col = ["BGD", "SGD","MBGD"]
BGD_MSE = np.average(np.array(BGD_MSE), 0)
SGD_MSE = np.average(np.array(SGD_MSE), 0)
MBGD_MSE = np.average(np.array(MBGD_MSE), 0)
xticks = np.arange(1, len(BGD_MSE) + 1)
error = [BGD_MSE,SGD_MSE,MBGD_MSE]
error_result = Merge(error, col)
error_result.to_excel("./MSE_iteration=%d_learn_rate=%f.xlsx"%(Iteration,learn_rate))
error_result.describe().to_excel("./_iteration=%d_learn_rate=%f.xlsx"%(Iteration,learn_rate))
plt.plot(xticks, BGD_MSE, 'c-')
plt.plot(xticks, SGD_MSE, 'b-.')
plt.plot(xticks, MBGD_MSE, 'r--')
plt.legend(labels=col, loc="best")
plt.xlabel("Iterations")
plt.ylabel("Test MSE")
plt.xlim(1, len(BGD_MSE))
plt.grid(True)
plt.savefig("./MSE_iteration=%d_learn_rate=%f.jpg"%(Iteration,learn_rate))
plt.show()
plt.close()

# Calculate and visualize the test accuracy of the BP neural network model and the neural network model based on the intercellular environment
col = ["BGD", "SGD", "MBGD"]
BGD_Accuracy = np.average(np.array(BGD_Accuracy), 0)
SGD_Accuracy = np.average(np.array(SGD_Accuracy), 0)
MBGD_Accuracy = np.average(np.array(MBGD_Accuracy), 0)
acc = [BGD_Accuracy,SGD_Accuracy,MBGD_Accuracy]
accuracy = Merge(acc, col)
accuracy.to_excel("./_iteration=%d_learn_rate=%f.xlsx"%(Iteration,learn_rate))
accuracy.describe().to_excel("./_iteration=%d_learn_rate=%f.xlsx"%(Iteration,learn_rate))
xticks = np.arange(1, len(BGD_Accuracy) + 1)
plt.plot(xticks, BGD_Accuracy, 'c-')
plt.plot(xticks, SGD_Accuracy, 'b--')
plt.plot(xticks, MBGD_Accuracy, 'r-.')
plt.legend(labels=col, loc="best")
plt.xlabel("Iterations")
plt.ylabel("Test Accuracy")
plt.grid(True)
plt.savefig("./iteration=%d_learn_rate=%f.jpg"%(Iteration,learn_rate))
plt.show()
plt.close()


<ipython-input-17-d5ef9f9e78b7>:299: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.Train_Data = np.array(mini_batch_inputs)
<ipython-input-17-d5ef9f9e78b7>:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.Train_Label = np.array(mini_batch_outputs)


10,learn_rate=0.010000
BGD TA: 0.660000,MSE：0.680000
SGD TA: 0.650000,MSE：0.700000
MBGD TA：0.660000,MSE：0.680000
20,learn_rate=0.010000
BGD TA: 0.795000,MSE：0.410000
SGD TA: 0.755000,MSE：0.490000
MBGD TA：0.795000,MSE：0.410000
30,learn_rate=0.010000
BGD TA: 0.815000,MSE：0.370000
SGD TA: 0.795000,MSE：0.410000
MBGD TA：0.815000,MSE：0.370000
40,learn_rate=0.010000
BGD TA: 0.825000,MSE：0.350000
SGD TA: 0.810000,MSE：0.380000
MBGD TA：0.825000,MSE：0.350000
50,learn_rate=0.010000
BGD TA: 0.820000,MSE：0.360000
SGD TA: 0.810000,MSE：0.380000
MBGD TA：0.820000,MSE：0.360000
60,learn_rate=0.010000
BGD TA: 0.825000,MSE：0.350000
SGD TA: 0.810000,MSE：0.380000
MBGD TA：0.825000,MSE：0.350000
70,learn_rate=0.010000
BGD TA: 0.825000,MSE：0.350000
SGD TA: 0.810000,MSE：0.380000
MBGD TA：0.825000,MSE：0.350000
80,learn_rate=0.010000
BGD TA: 0.825000,MSE：0.350000
SGD TA: 0.820000,MSE：0.360000
MBGD TA：0.825000,MSE：0.350000
90,learn_rate=0.010000
BGD TA: 0.825000,MSE：0.350000
SGD TA: 0.825000,MSE：0.350000
MBGD TA：0.825

In [2]:
print(best_accr)

0


In [3]:
data["predict"]=best_model.test(np.array(data[features]))[:,1]

AttributeError: 'int' object has no attribute 'test'

In [4]:
data

NameError: name 'data' is not defined

In [5]:
data.to_csv("./result.csv",index=None)

NameError: name 'data' is not defined